In [34]:
# importing Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import keras
from keras.models import Sequential
from keras.layers import Dense

In [2]:
# importing data
dat1=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Deep_Learning/DATA/AQI.csv")

In [5]:
dat1.head(5)

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [4]:
dat1.shape

(1093, 9)

In [9]:
# variable split
x=dat1.drop("PM 2.5",axis=1)
y=dat1["PM 2.5"]

In [10]:
x.shape

(1093, 8)

In [11]:
x.head()

,T,TM,Tm,SLP,H,VV,V,VM
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2


In [13]:
# train Test Split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=50)

In [15]:
# standardizing the data points
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)


In [31]:
# Developing model Parameters
def model_building(hp):
  model=Sequential()
# Input layer
  model.add(Dense(unit=8,kernel_initializer="he_uniform",input_dim=8))

# Hidden Layers
  for i in range(hp.Int('num_layers', 2, 20)):
        model.add(Dense(units=hp.Int('units_'+str(i),
                                     min_value=2,
                                     max_value=32,
                                     step=2),
                        activation=hp.Choice("activation", ["relu", "tanh", "sigmoid"])))
# Output Layer
  model.add(Dense(units=1,activation="relu"))

# Compiling the model
  model.compile(optimizer=keras.optimizer.Adam(hp.choice("learning_rate",[1e-2,1e-3,1e-4])),
                loss=mean_squared_error,
                metrics=["accuracy"])
  return model

In [32]:
# import Keras tuner
!pip install keras-tuner
import keras_tuner
from keras_tuner import RandomSearch

In [36]:
tuner = RandomSearch(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=3,
    max_retries_per_trial=1,
    directory = 'my_dir',
    project_name='AQI')

In [39]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 32, 'step': 2, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 32, 'step': 2, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [41]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

In [42]:
tuner.results_summary()

Results summary
Results in my_dir/AQI
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
num_layers: 18
units_0: 18
activation: tanh
units_1: 18
learning_rate: 0.001
units_2: 2
units_3: 2
units_4: 2
units_5: 2
units_6: 2
units_7: 2
units_8: 2
units_9: 2
units_10: 2
units_11: 2
units_12: 2
units_13: 2
units_14: 2
units_15: 2
units_16: 2
units_17: 2
Score: 0.04566210135817528

Trial 1 summary
Hyperparameters:
num_layers: 10
units_0: 14
activation: relu
units_1: 30
learning_rate: 0.001
units_2: 24
units_3: 28
units_4: 32
units_5: 26
units_6: 8
units_7: 12
units_8: 26
units_9: 16
units_10: 2
units_11: 18
units_12: 8
units_13: 32
units_14: 4
units_15: 28
units_16: 6
units_17: 8
Score: 0.04566210135817528

Trial 2 summary
Hyperparameters:
num_layers: 14
units_0: 28
activation: relu
units_1: 6
learning_rate: 0.001
units_2: 32
units_3: 24
units_4: 12
units_5: 24
units_6: 24
units_7: 16
units_8: 8
units_9: 20
units_10: 16
units_11: 24
units_